#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo
from datetime import datetime

In [2]:
client = pymongo.MongoClient("mongo")
database = client.exercices
collection = database.kickstarter

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

- Dans la colonne "goal", il y a une date au milieu des flottants (ligne 66141)
- Dans la colonne "pledged", il y a une date au milieu des flottants (ligne 66141) 
- Dans la colonne "backers", il y a un mot au milieu des entiers (ligne 66141)
- Dans la colonne "usd pledged", il y a un mot au milieu des flottants (ligne 66141)        

Toutes les erreurs de conversion sont dues aux anomalies de la ligne 66141.       
On règle le problème en supprimant cette ligne.

In [5]:
print(df_ks.shape)
df_ks=df_ks.query("index!=66141")
df_ks.shape

(150000, 14)


(149999, 14)

On modifie le nom de la variable "ID" en "_id" afin d'utiliser cette colonne comme identifants des documents.

In [6]:
df_ks.ID.value_counts().max()==1

True

In [7]:
df_ks.columns=[col if col!="ID" else "_id" for col in df_ks.columns]
df_ks.columns

Index(['_id', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd pledged', 'usd_pledged_real'],
      dtype='object')

On convertit les variables anciennement concernées par des anomalies en entiers ou flottants.                        
Les variables launched et deadline sont mises au format datetime.

In [8]:
df_ks['_id']=df_ks['_id'].astype('float')
df_ks['goal']=df_ks['goal'].astype('float')
df_ks['backers']=df_ks['backers'].astype('int')
df_ks['pledged']=df_ks['pledged'].astype('float')
df_ks['usd pledged']=df_ks['usd pledged'].astype('float')
df_ks['launched']=df_ks['launched'].astype('datetime64[ns]')
df_ks['deadline']=df_ks['deadline'].astype('datetime64[ns]')
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149999 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   _id               149999 non-null  float64       
 1   name              149997 non-null  object        
 2   category          149999 non-null  object        
 3   main_category     149999 non-null  object        
 4   currency          149999 non-null  object        
 5   deadline          149999 non-null  datetime64[ns]
 6   goal              149999 non-null  float64       
 7   launched          149999 non-null  datetime64[ns]
 8   pledged           149999 non-null  float64       
 9   state             149999 non-null  object        
 10  backers           149999 non-null  int64         
 11  country           149999 non-null  object        
 12  usd pledged       148517 non-null  float64       
 13  usd_pledged_real  149999 non-null  float64       
dtypes: d

### Importer les données

On transforme chaque ligne de df_ks en dictionnaire avec la fonction suivante.

In [9]:
def to_documents(df,col_list='all'):
    """
    Transformation des lignes du dataframe df en 
    plusieurs dictionnaires

    Args:
        df: nom du dataframe à convertir
        col_list: liste des variables à exporter

    Returns:
        liste de dictionnaires issus de la conversion
    """
    if type(col_list)==type([]) and len(col_list)!=0:
        df=df.copy()[col_list]
    DOCUMENTS=df.to_dict(orient='records')
    return DOCUMENTS

DOCUMENTS=to_documents(df_ks,["_id","pledged","name","category","country","goal","launched"])
print(len(DOCUMENTS),DOCUMENTS[0],sep="\n")

149999
{'_id': 872782264.0, 'pledged': 1145.0, 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)', 'category': 'Rock', 'country': 'US', 'goal': 2000.0, 'launched': Timestamp('2011-08-17 06:31:31')}


On insère ensuite les dictionnaires dans la collection.

In [10]:
collection.drop()
collection.insert_many(DOCUMENTS)
collection.find_one()

{'_id': 872782264.0,
 'pledged': 1145.0,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'country': 'US',
 'goal': 2000.0,
 'launched': datetime.datetime(2011, 8, 17, 6, 31, 31)}

## Question 1  

On récupère les projets ayant reçut le plus de 5 promesses de dons.                
Pour cela, on fait un tri décroissant de la collection par rapport à pledged puis on revoie les 5 premiers documents.          
La requête est la suivante.

In [11]:
question1=collection.find({},{"name":1}).sort([("pledged", -1)]).limit(5)
question1=[project["name"] for project in list(question1)]
print(question1)
question1==list(df_ks.sort_values("pledged",ascending=False).head().name)

["COOLEST COOLER: 21st Century Cooler that's Actually Cooler", 'Pebble 2, Time 2 + All-New Pebble Core', 'Expect the Unexpected. digiFilmï¿½ Camera by YASHICA', 'OUYA: A New Kind of Video Game Console', 'The Everyday Backpack, Tote, and Sling']


True

## Question 2

On compte le nombre de projet ayant atteint leur but.                        
On sélectionne les documents ayant pledged>=goal puis on les dénombre avec la requête suivante.

In [12]:
question2=collection.find({"$where": "this.pledged>=this.goal" })
question2=len(list(question2))
print(question2)
question2==len(df_ks.query("pledged>=goal"))

54264


True

## Question 3
On compte le nombre de projet par catégorie.                                      
On regroupe les documents par category puis on les dénombre avec la requête suivante.

In [13]:
question3=collection.aggregate([{"$group" : {"_id" : "$category", "projectNumberByCategory" : {"$sum" : 1}}}])
question3=dict([(project["_id"],project["projectNumberByCategory"]) for project in list(question3)])

good_answer=df_ks.category.value_counts()
print(good_answer,end="\n")
verif=True
for cat in question3.keys():
    if question3[cat]!=good_answer.loc[cat]:
        verif=False
        break
print(verif)

Product Design     8886
Documentary        6498
Music              6229
Tabletop Games     5581
Shorts             4857
                   ... 
Quilts               32
Letterpress          24
Literary Spaces      10
Chiptune             10
Taxidermy             7
Name: category, Length: 159, dtype: int64
True


## Question 4
On compte le nombre de projets français ayant été instanciés avant 2016.                       
On sélectionne les documents ayant lauched.year<2016 et country=='FR' puis on les dénombre avec la requête suivante.

In [14]:
end = datetime(2015, 12, 31, 23, 59, 59)
question4=collection.find({"$and":[{"country":"FR"},{"launched": {"$lte": end}}]},{"launched":1})
question4=len(list(question4))
print(question4)

good_answer=len((df_ks[df_ks.launched.apply(lambda x: x.year<2016)]).query("country=='FR'"))
question4==good_answer

330


True

## Question 5

On récupère les projets américains ayant demandé plus de 200 000 dollars.                      
On sélectionne les documents ayant goal>200000 et country=='US' avec la requête suivante.

In [15]:
question5=collection.find({"$and":[{"goal":{"$gt":200000}},{"country":"US"}]},{"name":1,"goal":1})
question5=[project["name"] for project in list(question5)]
print(question5[0:10]+["..."])

question5==list(df_ks.query("country=='US' & goal>200000").name)

['A CALL TO ADVENTURE', 'Storybricks, the storytelling online RPG', 'Shine On New World', 'Nightclub', 'Nastaran (Wild Rose)', 'Hubo - Extension Box for iPhone', 'Baja ATV Park (Suspended)', 'Chihuly Installation for Orlando (Pulse Nightclub)', 'Kurt Vonnegut: Unstuck in Time', 'The LAKE HOPPER is a VTOL Flying Water Craft Made in America', '...']


True

## Question 6

On compte le nombre de projet ayant "Sport" dans leur nom.                              
On crée un index textuel sur name puis on sélectionne les documents comportant 'Sport' dans name avec l'opérateur $text dans la requête suivante.

In [16]:
collection.create_index([("name","text")])
question6=collection.find({"$text": {"$search": "Sport"}},{"name":1})
question6=[projet["name"] for projet in list(question6)]
print(question6[0:20]+["..."],len(question6),sep=2*"\n")

['Sport Smart. A New Genre of Sports TV. Sport Fans Unite!!!', 'Frey Sports App - We connect sports people.', 'Daily Fantasy Sports | Sports Analytics Platform | DFS', 'Intro To: Sports Tech and THE Sports Techie Online Community', 'A New Sport Show For The New Generation of Sports Fans!', 'Odds Hacker - Sports Betting and Fantasy Sports Predictions', 'Elite-Sports-Worldwide- presents all around sports socks!', 'sport to go: Le sport partout et sans engagement!', 'S2SA - Sport to Strap Adapter for Samsung Gear S2 Sport (3G)', "Sports Liberated l Free women's sport lifestyle magazine app", 'Zumer Sport:Lunch Boxes Made From Actual Sport Ball Material', 'Sports podcast', 'Skyline Sports', 'Sports Court', 'Sport Contender', 'The Sport of Photography', 'Sports And The City', 'Sports Stream', 'Universal sports', 'Utility Sports', '...']

318
